In [1]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder  
import numpy as np
import pandas as pd

In [18]:
class DataSet(object):
    def __init__(self, csv_path):
        self.data = pd.read_csv(csv_path)
        self.processed_data = self.data
        self.row_num = len(self.data)
        self.col_num = len(self.data.columns)
        #self.missing_value_process()

        #self.raw_predictor = self.data.iloc[:,:-1].values
        #self.raw_response = self.data.iloc[:,-1].values

        #self.missing_value_process()
        self.tag_transfer()
    
    def missing_value_process(self):
        for col in range(self.col_num - 1):
            qm_num = 0
            for row in range(self.row_num - 1):
                if str(self.data.iat[row,col]) == "?":
                    qm_num += 1

            print(qm_num/self.row_num)
            if qm_num/self.row_num >= 0.5:
                print("true")
                self.processed_data.drop(self.processed_data.columns[[col]], axis=1, inplace = True)

    def tag_transfer(self): 

        le = LabelEncoder()

        # 遍历DataFrame的每一列
        for column in self.processed_data.columns:
            # 如果列的数据类型是对象（通常意味着它是分类类型）
            if self.processed_data[column].dtype == 'object':
                # 对列应用 LabelEncoder
                self.processed_data[column] = le.fit_transform(self.processed_data[column])


    #I have changed        

        #self.processed_data = self.raw_data.dropna()





In [19]:
train_data = DataSet('diabetic_data_training.csv')
test_data = DataSet('diabetic_data_test.csv')

#train_data.data
train_data.processed_data


/Users/yorux/anaconda3/envs/jupyter/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/yorux/anaconda3/envs/jupyter/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/yorux/anaconda3/envs/jupyter/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/yorux/anaconda3/envs/jupyter/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstan

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,149190,55629189,3,0,1,1,1,1,7,3,...,0,3,1,0,0,0,0,0,1,1
1,64410,86047875,1,0,2,1,1,1,7,2,...,0,1,1,0,0,0,0,1,1,2
2,500364,82442376,3,1,3,1,1,1,7,2,...,0,3,1,0,0,0,0,0,1,2
3,16680,42519267,3,1,4,1,1,1,7,1,...,0,2,1,0,0,0,0,0,1,2
4,35754,82637451,3,1,5,1,2,1,2,3,...,0,2,1,0,0,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91584,443847548,100162476,1,1,7,1,1,3,7,3,...,0,0,1,0,0,0,0,0,1,1
91585,443847782,74694222,1,0,8,1,1,4,5,5,...,0,2,1,0,0,0,0,1,1,2
91586,443854148,41088789,3,1,7,1,1,1,7,1,...,0,0,1,0,0,0,0,0,1,2
91587,443857166,31693671,3,0,8,1,2,3,7,10,...,0,3,1,0,0,0,0,0,1,2


In [8]:
train_data = pd.read_csv('diabetic_data_training.csv')
test_data = pd.read_csv('diabetic_data_test.csv')

predictor = train_data.iloc[:,:-1].values
response = train_data.iloc[:,-1].values

predictor,response 

(array([[149190, 55629189, 'Caucasian', ..., 'No', 'Ch', 'Yes'],
        [64410, 86047875, 'AfricanAmerican', ..., 'No', 'No', 'Yes'],
        [500364, 82442376, 'Caucasian', ..., 'No', 'Ch', 'Yes'],
        ...,
        [443854148, 41088789, 'Caucasian', ..., 'No', 'Ch', 'Yes'],
        [443857166, 31693671, 'Caucasian', ..., 'No', 'Ch', 'Yes'],
        [443867222, 175429310, 'Caucasian', ..., 'No', 'No', 'No']],
       dtype=object),
 array(['>30', 'NO', 'NO', ..., 'NO', 'NO', 'NO'], dtype=object))